## Домашнее задание 3: Contingency Table

### О задании

1) Consider three nominal features (one of them, not more, may be taken from nominal features in your data).

2) Build two contingency tables over them: present a conditional frequency table and Quetelet relative index tables. Make comments on relations between categories of the common (to both tables) feature and two others.

3) Compute and visualize the chi-square-summary_Quetelet_index over both tables.

4) Comment on the meaning of the values in the data analysis context.

In [188]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Загрузим данные

In [232]:
df = pd.read_csv('../../data/Vehicles03.csv')
df.head()

,Vehicle_Type,Sex_of_Driver,Age_of_Driver
0,9,2,74.0
1,11,1,42.0
2,11,1,35.0
3,9,1,62.0
4,9,2,49.0


Добавим 2 номинальных признака

In [225]:
# Motorcylce, vehilce type should be >=2 and <= 5
df = df.assign(Is_Motorcylce=df.Vehicle_Type.copy())
df["Is_Motorcylce"] = 1
df.loc[df["Vehicle_Type"]<2, "Is_Motorcylce"] = 0
df.loc[df["Vehicle_Type"]>5, "Is_Motorcylce"] = 0
df.loc[df["Vehicle_Type"]==-1, "Is_Motorcylce"] = 0

del df["Vehicle_Type"]

# Driver is younger than 40
df = df.assign(Age_of_Driver_LEQ40=df.Age_of_Driver.copy())
df.loc[df["Age_of_Driver"]<=40, "Age_of_Driver_LEQ40"] = 1
df.loc[df["Age_of_Driver"]>40, "Age_of_Driver_LEQ40"] = 0
df.head()

del df["Age_of_Driver"]
df.head()

,Sex_of_Driver,Is_Motorcylce,Age_of_Driver_LEQ40
0,2,0,0.0
1,1,0,0.0
2,1,0,1.0
3,1,0,0.0
4,2,0,0.0


In [226]:
df.columns

Index(['Sex_of_Driver', 'Is_Motorcylce', 'Age_of_Driver_LEQ40'], dtype='object')

В качестве 3-х номинальных признаков выберем: "Sex_of_Driver", "Is_Motorcylce", "Age_of_Driver_LEQ40"

Построим conditional frequency table для признака "Sex_of_Driver"

In [227]:
import tabletext

In [228]:
a = [["Sex_of_Driver/Is_Motorcylce", "True", "False"], ["Man"], ["Woman"], ["Unknown"]]

total = df.shape[0] * df.shape[1]
for i in range(1, 4):
    for j in range(2):
        x = ((df["Sex_of_Driver"] == i) & (df["Is_Motorcylce"] == j)).sum()
        a[i].append(str(x / total * 100)[0:8] + "%")

print(tabletext.to_text(a))

┌─────────────────────────────┬───────────┬───────────┐
│ Sex_of_Driver/Is_Motorcylce │ True      │ False     │
├─────────────────────────────┼───────────┼───────────┤
│ Man                         │ 20.74863% │ 4.094298% │
├─────────────────────────────┼───────────┼───────────┤
│ Woman                       │ 8.094230% │ 0.365562% │
├─────────────────────────────┼───────────┼───────────┤
│ Unknown                     │ 0.028904% │ 0.001700% │
└─────────────────────────────┴───────────┴───────────┘


In [229]:
a = [["Sex_of_Driver/Age_of_Driver_LEQ40", "True", "False"], ["Man"], ["Woman"], ["Unknown"]]
for i in range(1, 4):
    for j in range(2):
        x = ((df["Sex_of_Driver"] == i) & (df["Age_of_Driver_LEQ40"] == j)).sum()
        a[i].append(str(x / total * 100)[0:8] + "%")

print(tabletext.to_text(a))

┌───────────────────────────────────┬───────────┬───────────┐
│ Sex_of_Driver/Age_of_Driver_LEQ40 │ True      │ False     │
├───────────────────────────────────┼───────────┼───────────┤
│ Man                               │ 9.122055% │ 15.72088% │
├───────────────────────────────────┼───────────┼───────────┤
│ Woman                             │ 3.069023% │ 5.390769% │
├───────────────────────────────────┼───────────┼───────────┤
│ Unknown                           │ 0.010201% │ 0.020403% │
└───────────────────────────────────┴───────────┴───────────┘


Построим Quetelet relative index table для "Is_Motorcylce"

In [230]:
a = [["Is_Motorcylce/Sex_of_Driver", "Man", "Woman", "Unknown"], ["True"], ["False"]]
for i in range(1, 3):
    for j in range(1, 4):
        x = ((df["Is_Motorcylce"] == i - 1) & (df["Sex_of_Driver"] == j)).sum() / total
        x /= (df["Is_Motorcylce"] == i - 1).sum() / total
        y = (df["Sex_of_Driver"] == j).sum() / total
        a[i].append(str(((x - y) / y) * 100)[0:8] + "%")

print(tabletext.to_text(a))

┌─────────────────────────────┬───────────┬───────────┬───────────┐
│ Is_Motorcylce/Sex_of_Driver │ Man       │ Woman     │ Unknown   │
├─────────────────────────────┼───────────┼───────────┼───────────┤
│ True                        │ 189.2765% │ 231.3922% │ 227.1168% │
├─────────────────────────────┼───────────┼───────────┼───────────┤
│ False                       │ 269.3939% │ -3.14658% │ 24.52045% │
└─────────────────────────────┴───────────┴───────────┴───────────┘


Построим Quetelet relative index table для "Age_of_Driver_LEQ40"

In [231]:
a = [["Age_of_Driver_LEQ40/Sex_of_Driver", "Man", "Woman", "Unknown"], ["True"], ["False"]]
for i in range(1, 3):
    for j in range(1, 4):
        x = ((df["Age_of_Driver_LEQ40"] == i - 1) & (df["Sex_of_Driver"] == j)).sum() / total
        x /= (df["Age_of_Driver_LEQ40"] == i - 1).sum() / total
        y = (df["Sex_of_Driver"] == j).sum() / total
        a[i].append(str(((x - y) / y) * 100)[0:8] + "%")

print(tabletext.to_text(a))

┌───────────────────────────────────┬───────────┬───────────┬───────────┐
│ Age_of_Driver_LEQ40/Sex_of_Driver │ Man       │ Woman     │ Unknown   │
├───────────────────────────────────┼───────────┼───────────┼───────────┤
│ True                              │ 200.9430% │ 197.3274% │ 173.1953% │
├───────────────────────────────────┼───────────┼───────────┼───────────┤
│ False                             │ 199.4554% │ 201.5430% │ 215.4765% │
└───────────────────────────────────┴───────────┴───────────┴───────────┘
